In [ ]:
# This command can be used to install the required packages for this project.
# !pip install -r ../requirements.txt

In [ ]:
# Add the parent directory of 'KernelCT' to sys.path

import sys
sys.path.append('..')

In [ ]:
# Import necessary packages/modules
import numpy as np
from KernelCT.object import Object
from KernelCT import utilities, phantom
from KernelCT.kernel_reconstruction import KernelRegressor, KernelInterpolant
from KernelCT.weighted_kernels import WeightedGaussian

In [ ]:
# Setup

# Define phantom
phantom_type = 'shepp_logan'

# Generate pixel grid
x_max = 1
y_max = 1
grid_size = [256, 256]
X, Y = utilities.generate_pixel_grid(x_max, y_max, grid_size)
phantom_eval = phantom.eval_phantom(X, Y, phantom_type)

# Create object
geometry = 'random'
r_max = np.sqrt(2)
sample_size = 10000

r, a = utilities.generate_lineset(geometry, r_max, sample_size)
Radon = phantom.eval_phantom_radon(r, a, phantom_type)
obj = Object(r, a, Radon)

In [ ]:
# Run greedy kernel thinning
kernel = WeightedGaussian()
greedy_method = 'beta_greedy'
max_iter = 1500
beta = 0.5
c_newton = None
V_newton = None
pwr_func_vals = None
res = None

c_newton, V_newton, pwr_func_vals, res = obj.kernel_thinning(
    kernel,
    greedy_method,
    max_iter=max_iter,
    beta=beta,
    c_newton=c_newton,
    V_newton=V_newton,
    pwr_func_vals=pwr_func_vals,
    res=res
)

In [ ]:
# Compare different regularization methods

# Initialize plot
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(24, 6))

# Plot original phantom
ax = fig.add_subplot(1, 4, 1)
ax.imshow(phantom_eval, cmap='gray', origin='lower')
ax.set_title('Original Phantom')
ax.axis('off')

# Compute interpolant
interpolant = KernelInterpolant()
eval_repr = None
interpolant.fit(
    obj, kernel, basis='newton', 
    c_newton=c_newton, V_newton=V_newton
)
reconstruction, eval_repr = interpolant.predict(
    obj, X, Y, kernel, prev_eval_repr=None
)

ax = fig.add_subplot(1, 4, 2)
ax.imshow(reconstruction, cmap='gray', origin='lower')
ax.set_title('Interpolation')
ax.axis('off')

# Initialize kernel regressor
regressor = KernelRegressor()

# Regularization with norm-based method
regressor.fit(
    obj, kernel, regularization='norm',
    gamma=1e-6, V_newton=V_newton
)
reconstruction, eval_repr = regressor.predict(
    obj, X, Y, kernel, eval_repr
)

ax = fig.add_subplot(1, 4, 3)
ax.imshow(reconstruction, cmap='gray', origin='lower')
ax.set_title('Norm regularization')
ax.axis('off')

# Regularization with TV-based method
regressor.fit(
    obj, kernel, regularization='tv',
    gamma=1e-10, X_diff=X, Y_diff=Y
)
reconstruction, eval_repr = regressor.predict(
    obj, X, Y, kernel, eval_repr
)

ax = fig.add_subplot(1, 4, 4)
ax.imshow(reconstruction, cmap='gray', origin='lower')
ax.set_title('TV regularization')
ax.axis('off')

plt.show()